<a href="https://www.kaggle.com/code/emmermarcell/create-a-wikipedia-corpus?scriptVersionId=159940263" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Create a Wikipedia corpus

The aim of this notebook is to preprocess an existing wikipedia dump and create a wiikpedia corpus can can be used for NLP tasks. Since the dataset is quite large I will make use of the memory mapping between the RAM and the filesystems storage done by the the [Hugging Face Datasets library][1]. Under the hood, it utilizes the Apache Arrow memory format and pyarrow library. Unfortunately the [`wikipedia`][8] dataset is not streamable so I stick to iterating through it.

[1]: https://huggingface.co/learn/nlp-course/chapter5/4?fw=pt
[8]: https://huggingface.co/datasets/wikipedia

In [1]:
!pip install blingfire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 7.9 MB/s eta 0:00:00:00:0100:01


In [2]:
import os
import glob
import gzip
import gc    # Garbage collector
from tqdm.auto import tqdm

from blingfire import text_to_sentences
import numpy as np
from datasets import load_dataset

The `all-MiniLM-L6-v2` model can handle a maximum sequence size of 256 words. As Wikipedia articles are often longer, I use the `text_to_sentences` function from`blingfire` to preprocess the data. This function takes a string (representing a Wikipedia article) and breaks up the article by sentences. The sentences are then saved into the `wikipedia_processed_*.txt` files locally line-by-line. After, I can simply utilize the Huggingface datasets library. 

In [3]:
def process_wikipedia_dataset(wiki_dataset, output_dir, articles_per_file=1_000_000):
    file_count = 1
    article_count = 0

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Open the first output file for writing in compressed format
    out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')

    # Iterate through each article in the dataset
    for article in tqdm(wiki_dataset, desc='Processing Articles'):
        # Process each line of the article into sentences
        for line in article['text'].split('\n'):
            sentences = text_to_sentences(line)
            out_f.write(sentences + '\n')

        article_count += 1
        # Check if it's time to switch to a new file
        if article_count >= articles_per_file:
            out_f.close()
            file_count += 1
            article_count = 0
            out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')

    # Close the last file
    out_f.close()

    # Pattern to match the compressed files
    processed_file_paths = f"{output_dir}/wikipedia_processed_*.txt.gz"
    # List all compressed files matching the pattern
    processed_file_list = glob.glob(processed_file_paths)
    print(f'The processed files are: {processed_file_list}')
    
    return processed_file_list

In [4]:
# Load the Wikipedia dataset
wiki_dataset = load_dataset("wikipedia", "20220301.en", split='train')
print(f'Length of the Wikipedia dataset is {len(wiki_dataset)} articles.')

# Specify the working directory to save the processed files
output_dir = '/kaggle/working'
# Process the Wikipedia dataset and return the path to the processed files
processed_file_list = process_wikipedia_dataset(wiki_dataset, output_dir)

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to /root/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.
Length of the Wikipedia dataset is 6458670 articles.


Processing Articles:   0%|          | 0/6458670 [00:00<?, ?it/s]

The processed files are: ['/kaggle/working/wikipedia_processed_2.txt.gz', '/kaggle/working/wikipedia_processed_3.txt.gz', '/kaggle/working/wikipedia_processed_5.txt.gz', '/kaggle/working/wikipedia_processed_1.txt.gz', '/kaggle/working/wikipedia_processed_6.txt.gz', '/kaggle/working/wikipedia_processed_7.txt.gz', '/kaggle/working/wikipedia_processed_4.txt.gz']


### The resulting files can be saved in a Kaggle dataset.